In [ ]:
import os
import pandas as pd
%matplotlib inline
%pylab inline
pylab.rcParams['figure.figsize']=(14,6)
# import necessary libraries and specify that graphs should be plotted inline.
import numpy as np
import matplotlib.pyplot as plt
import math
from matplotlib.colors import ListedColormap
from sklearn import neighbors, datasets
from sklearn.cross_validation import train_test_split,cross_val_score,StratifiedKFold, KFold
#from sklearn.model_selection import cross_val_score
from sklearn import feature_selection
from sklearn import datasets, tree, linear_model, svm, cross_validation
from sklearn import preprocessing
from sklearn.metrics import precision_score, \
    recall_score, confusion_matrix, classification_report, \
    accuracy_score, f1_score, make_scorer, roc_auc_score, roc_curve
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.feature_selection import plot_sequential_feature_selection as plot_sfs
import matplotlib.pyplot as plt
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_digits
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from __future__ import print_function
import subprocess
import time
from operator import itemgetter
import random
random.seed(0)
import numpy as np
np.random.seed(0)
min_max_scaler = preprocessing.MinMaxScaler()
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTEENN,SMOTETomek
from pyxdameraulevenshtein import damerau_levenshtein_distance, normalized_damerau_levenshtein_distance,\
damerau_levenshtein_distance_ndarray, normalized_damerau_levenshtein_distance_ndarray
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
import warnings
import re
import gensim, logging
from scipy import spatial
warnings.filterwarnings('ignore')
from sklearn.cross_validation import train_test_split,cross_val_score,StratifiedKFold, KFold



In [ ]:
desc_clean=pd.read_csv("description_cleaned_lemma.csv")
product_train=pd.read_csv("product_train_lemma.csv")
desc_clean["product_description"] = desc_clean["product_description"].astype(str)
desc_clean["product_brand"] = desc_clean["product_brand"].astype(str)
desc_clean["product_color"] = desc_clean["product_color"].astype(str)
desc_clean["product_attribute_concat"] = desc_clean["product_attribute_concat"].astype(str)

product_train["product_title"] = product_train["product_title"].astype(str)
product_train["search_term"] = product_train["search_term"].astype(str)


In [ ]:
from ggplot import *
p = ggplot(aes(x='relevance'), data=product_train)
p + geom_histogram()


from ggplot import *
#features["relevance"]
p = ggplot(aes(x='relevance'), data=product_train)
p + geom_histogram(colour="#FF9999",fill = "black")+xlab("Relevance (Target)") +ylab("Count") + ggtitle("Population Distribution")

p + theme_bw()


In [ ]:
X = product_train["search_term"] # we only take the first two features in order to easily visualize the results. 
                      # We could avoid this ugly slicing by using a two-dim dataset
y = product_train["relevance"]

In [ ]:
X_waste, X_sample, y_waste, y_sample = train_test_split(
     X, y, test_size=0.01, random_state=0,stratify=y)

X_sample.shape

In [ ]:
unique = pd.DataFrame(X_sample.unique(), columns=['search_term'])
product_train=pd.merge(product_train,unique)

In [ ]:
from ggplot import *
p = ggplot(aes(x='relevance'), data=product_train)
p + geom_histogram()

In [ ]:
#Functions
def fuzzymatch(x,y,edits):
    a = x.split()
    b = y.split()
    match = 0
    
    for i in range(0,len(a)):
        for j in range(0,len(b)):
            if damerau_levenshtein_distance(a[i],  b[j]) <= edits:
                match = match + 1
                j = j + 1
    return(match)

def uniquefuzzymatch(x,y,edits):
    a = x.split()
    b = y.split()
    match = 0
    
    for i in range(0,len(a)):
        counter=0
        for j in range(0,len(b)):
            if damerau_levenshtein_distance(a[i],  b[j]) <= edits:
                if counter==0:
                    match = match + 1
                    counter=counter+1
                j = j + 1
    return(match)



def position_match(x,y,pos,edits):
    a = x.split()
    b = y.split()
    if pos=="first":
        word=a[0]
    else:
        word=a[len(a)-1]
    position = 0
    counter=0
    for j in range(0,len(b)):
        if damerau_levenshtein_distance(word,  b[j]) <= edits:
            if counter==0:
                position=j
                counter=1
            
    return(position)

def match_query(x,y):
    if x in y:
        return(1)
    else:
        return(0)

def jacc_dist(x,y,edits):
    num=uniquefuzzymatch(x,y,edits)
    den=len(x.split())+len(y.split())-num
    return(float(num)/float(den))

def dice_dist(x,y,edits):
    num=uniquefuzzymatch(x,y,edits)
    den=len(x.split())+len(y.split())
    return(float(2*num)/float(den))

def cosine_dist(x,y,edits):
    num=uniquefuzzymatch(x,y,edits)
    den=math.sqrt(len(x.split()))*math.sqrt(len(y.split()))
    return(float(2*num)/float(den))


def makeFeatureVec(frame, model, num_features,algo):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    words=frame.split()
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed
    if algo=="word2vec":
        index2word_set = set(model.index2word)
    else:
        index2word_set=vectorizer.get_feature_names()
        lsa_matrix=pd.DataFrame(lsa.components_,columns =vectorizer.get_feature_names())
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            if algo=="word2vec":
                featureVec = np.add(featureVec,model[word])
            else:
                featureVec = np.add(featureVec,lsa_matrix[word].values)
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec



def tfidfFeatureVec(frame, model, num_features,tfidf_dict,algo):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    words=frame.split()
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    if algo=="word2vec":
        index2word_set = set(model.index2word)
    else:
        index2word_set=vectorizer.get_feature_names()
        lsa_matrix=pd.DataFrame(lsa.components_,columns =vectorizer.get_feature_names())

    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set and word in tfidf_dict.keys(): 
            nwords = nwords + 1.
            if algo=="word2vec":
                featureVec = np.add(featureVec,model[word]*tfidf_dict[word])
            else:
                featureVec = np.add(featureVec,lsa_matrix[word].values*tfidf_dict[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec
    

def ngram_count(text1, text2,n):
    count= 0
    trigrams1 = ngrams(text1.split(), n)
    for grams1 in trigrams1:
        trigrams2 = ngrams(text2.split(), n)
        for grams2 in trigrams2:
            if n==2:
                if damerau_levenshtein_distance(grams1[0],grams2[0])<=2 and damerau_levenshtein_distance(grams1[1],grams2[1])<=2:
                    count=count+1
            else:
                if damerau_levenshtein_distance(grams1[0],grams2[0])<=2 and damerau_levenshtein_distance(grams1[1],grams2[1])<=2\
                and damerau_levenshtein_distance(grams1[2],grams2[2])<=2:
                    count=count+1
    return count


In [ ]:
prod_desc = pd.merge(product_train, desc_clean, on="product_uid")
#prod_desc=prod_desc.head()

#Calculate tfidf 
vectorizer = TfidfVectorizer(min_df = 1)
vectorizer.fit_transform(desc_clean["product_description"])
tfidf_dict={t[0]:t[1] for t in vectorizer.vocabulary_.items()}


#Make description corpus
corpus=[]
for i in range(shape(desc_clean)[0]):
    corpus.append(desc_clean["product_description"][i].split(" "))
model = gensim.models.Word2Vec(corpus, size=100,window=5, min_count=5,seed=0)



In [ ]:
vectorizer = TfidfVectorizer(min_df = 1,max_features=1000)
dtm = vectorizer.fit_transform(desc_clean["product_description"])
lsa = TruncatedSVD(100, algorithm = 'randomized',random_state=0)
dtm_lsa = lsa.fit_transform(dtm)
dtm_lsa = Normalizer(copy=False).fit_transform(dtm_lsa)
name=range(1,101)
names=['lsa_'+str(col)   for col in name]
lsa_matrix=pd.DataFrame(lsa.components_,index=names,columns =vectorizer.get_feature_names())

In [ ]:
import nltk
from nltk.util import ngrams

prod_desc["fuzzymatch_desc"]=prod_desc.apply(lambda x: fuzzymatch(x['search_term'],x['product_description'],2),axis=1)
prod_desc["match_desc"]=prod_desc.apply(lambda x: fuzzymatch(x['search_term'],x['product_description'],0),axis=1)
prod_desc["uniquefuzzymatch_desc"]=prod_desc.apply(lambda x: uniquefuzzymatch(x['search_term'],x['product_description'],2),axis=1)
prod_desc["uniquematch_desc"]=prod_desc.apply(lambda x: uniquefuzzymatch(x['search_term'],x['product_description'],0),axis=1)

prod_desc["fuzzymatch_title"]=prod_desc.apply(lambda x: fuzzymatch(x['search_term'],x['product_title'],2),axis=1)
prod_desc["match_title"]=prod_desc.apply(lambda x: fuzzymatch(x['search_term'],x['product_title'],0),axis=1)
prod_desc["uniquefuzzymatch_title"]=prod_desc.apply(lambda x: uniquefuzzymatch(x['search_term'],x['product_title'],2),axis=1)
prod_desc["uniquematch_title"]=prod_desc.apply(lambda x: uniquefuzzymatch(x['search_term'],x['product_title'],0),axis=1)

prod_desc["fuzzymatch_attr"]=prod_desc.apply(lambda x: fuzzymatch(x['search_term'],x['product_attribute_concat'],2),axis=1)
prod_desc["match_attr"]=prod_desc.apply(lambda x: fuzzymatch(x['search_term'],x['product_attribute_concat'],0),axis=1)
prod_desc["uniquefuzzymatch_attr"]=prod_desc.apply(lambda x: uniquefuzzymatch(x['search_term'],x['product_attribute_concat'],2),axis=1)
prod_desc["uniquematch_attr"]=prod_desc.apply(lambda x: uniquefuzzymatch(x['search_term'],x['product_attribute_concat'],0),axis=1)

#First position as most of the queries are color (attributes)
prod_desc["first_pos_desc_exact"]=prod_desc.apply(lambda x: position_match(x['search_term'],x['product_description'],"first",0),axis=1)
prod_desc["first_pos_title_exact"]=prod_desc.apply(lambda x: position_match(x['search_term'],x['product_title'],"first",0),axis=1)
prod_desc["first_pos_brand_exact"]=prod_desc.apply(lambda x: position_match(x['search_term'],x['product_brand'],"first",0),axis=1)
prod_desc["first_pos_color_exact"]=prod_desc.apply(lambda x: position_match(x['search_term'],x['product_color'],"first",0),axis=1)
prod_desc["first_pos_attr_exact"]=prod_desc.apply(lambda x: position_match(x['search_term'],x['product_attribute_concat'],\
                                                                 "first",0),axis=1)

prod_desc["first_pos_desc_fuzzy"]=prod_desc.apply(lambda x: position_match(x['search_term'],x['product_description'],"first",2),axis=1)
prod_desc["first_pos_title_fuzzy"]=prod_desc.apply(lambda x: position_match(x['search_term'],x['product_title'],"first",2),axis=1)
prod_desc["first_pos_brand_fuzzy"]=prod_desc.apply(lambda x: position_match(x['search_term'],x['product_brand'],"first",2),axis=1)
prod_desc["first_pos_color_fuzzy"]=prod_desc.apply(lambda x: position_match(x['search_term'],x['product_color'],"first",2),axis=1)
prod_desc["first_pos_attr_fuzzy"]=prod_desc.apply(lambda x: position_match(x['search_term'],x['product_attribute_concat'],\
                                                                 "first",2),axis=1)

#Last position with desc and title as most of the queries have last word as product
prod_desc["last_pos_desc_exact"]=prod_desc.apply(lambda x: position_match(x['search_term'],x['product_description'],"last",0),axis=1)
prod_desc["last_pos_title_exact"]=prod_desc.apply(lambda x: position_match(x['search_term'],x['product_title'],"last",0),axis=1)
prod_desc["last_pos_brand_exact"]=prod_desc.apply(lambda x: position_match(x['search_term'],x['product_brand'],"last",0),axis=1)
prod_desc["last_pos_color_exact"]=prod_desc.apply(lambda x: position_match(x['search_term'],x['product_color'],"last",0),axis=1)
prod_desc["last_pos_attr_exact"]=prod_desc.apply(lambda x: position_match(x['search_term'],x['product_attribute_concat'],"last",0),axis=1)

prod_desc["last_pos_desc_fuzzy"]=prod_desc.apply(lambda x: position_match(x['search_term'],x['product_description'],"last",2),axis=1)
prod_desc["last_pos_title_fuzzy"]=prod_desc.apply(lambda x: position_match(x['search_term'],x['product_title'],"last",2),axis=1)
prod_desc["last_pos_brand_fuzzy"]=prod_desc.apply(lambda x: position_match(x['search_term'],x['product_brand'],"last",2),axis=1)
prod_desc["last_pos_color_fuzzy"]=prod_desc.apply(lambda x: position_match(x['search_term'],x['product_color'],"last",2),axis=1)
prod_desc["last_pos_attr_fuzzy"]=prod_desc.apply(lambda x: position_match(x['search_term'],x['product_attribute_concat'],"last",2),axis=1)

#Match the whole search query
prod_desc["query_match_desc"]=prod_desc.apply(lambda x: match_query(x['search_term'],x['product_description']),axis=1)
prod_desc["query_match_title"]=prod_desc.apply(lambda x: match_query(x['search_term'],x['product_title']),axis=1)
prod_desc["query_match_attr"]=prod_desc.apply(lambda x: match_query(x['search_term'],x['product_attribute_concat']),axis=1)

#Distances
prod_desc["jacc_title_exact"]=prod_desc.apply(lambda x: jacc_dist(x['search_term'],x['product_title'],0),axis=1)
prod_desc["dice_title_exact"]=prod_desc.apply(lambda x: dice_dist(x['search_term'],x['product_title'],0),axis=1)
prod_desc["cosine_title_exact"]=prod_desc.apply(lambda x: cosine_dist(x['search_term'],x['product_title'],0),axis=1)
prod_desc["jacc_title_fuzzy"]=prod_desc.apply(lambda x: jacc_dist(x['search_term'],x['product_title'],2),axis=1)
prod_desc["dice_title_fuzzy"]=prod_desc.apply(lambda x: dice_dist(x['search_term'],x['product_title'],2),axis=1)
prod_desc["cosine_title_fuzzy"]=prod_desc.apply(lambda x: cosine_dist(x['search_term'],x['product_title'],2),axis=1)

prod_desc["jacc_desc_exact"]=prod_desc.apply(lambda x: jacc_dist(x['search_term'],x['product_description'],0),axis=1)
prod_desc["dice_desc_exact"]=prod_desc.apply(lambda x: dice_dist(x['search_term'],x['product_description'],0),axis=1)
prod_desc["cosine_desc_exact"]=prod_desc.apply(lambda x: cosine_dist(x['search_term'],x['product_description'],0),axis=1)
prod_desc["jacc_desc_fuzzy"]=prod_desc.apply(lambda x: jacc_dist(x['search_term'],x['product_description'],2),axis=1)
prod_desc["dice_desc_fuzzy"]=prod_desc.apply(lambda x: dice_dist(x['search_term'],x['product_description'],2),axis=1)
prod_desc["cosine_desc_fuzzy"]=prod_desc.apply(lambda x: cosine_dist(x['search_term'],x['product_description'],2),axis=1)


prod_desc["jacc_attr_exact"]=prod_desc.apply(lambda x: jacc_dist(x['search_term'],x['product_attribute_concat'],0),axis=1)
prod_desc["dice_attr_exact"]=prod_desc.apply(lambda x: dice_dist(x['search_term'],x['product_attribute_concat'],0),axis=1)
prod_desc["cosine_attr_exact"]=prod_desc.apply(lambda x: cosine_dist(x['search_term'],x['product_attribute_concat'],0),axis=1)
prod_desc["jacc_attr_fuzzy"]=prod_desc.apply(lambda x: jacc_dist(x['search_term'],x['product_attribute_concat'],2),axis=1)
prod_desc["dice_attr_fuzzy"]=prod_desc.apply(lambda x: dice_dist(x['search_term'],x['product_attribute_concat'],2),axis=1)
prod_desc["cosine_attr_fuzzy"]=prod_desc.apply(lambda x: cosine_dist(x['search_term'],x['product_attribute_concat'],2),axis=1)




In [ ]:
#word2vec vectors
prod_desc['title_word_vector'] = prod_desc['product_title'].apply(lambda x: makeFeatureVec(x,model,100,"word2vec"))
prod_desc['desc_word_vector'] = prod_desc['product_description'].apply(lambda x: makeFeatureVec(x,model,100,"word2vec"))
prod_desc['search_word_vector'] = prod_desc['search_term'].apply(lambda x: makeFeatureVec(x,model,100,"word2vec"))
prod_desc['attr_word_vector'] = prod_desc['product_attribute_concat'].apply(lambda x: makeFeatureVec(x,model,100,"word2vec"))



In [ ]:
for i in range(0,len(prod_desc['search_term'])):
    if np.isnan(prod_desc["search_word_vector"][i][1]):
        prod_desc.drop(i,inplace=True)
prod_desc = prod_desc.reset_index(drop=True)


for i in range(0,len(prod_desc['search_term'])):
    if np.isnan(prod_desc["attr_word_vector"][i][1]):
        prod_desc.drop(i,inplace=True)
prod_desc = prod_desc.reset_index(drop=True)

In [ ]:

prod_desc['cos_title_word'] = prod_desc.apply(lambda x: spatial.distance.cosine(x["search_word_vector"],x["title_word_vector"]),\
                                              axis=1)
prod_desc['cos_desc_word'] = prod_desc.apply(lambda x: spatial.distance.cosine(x["search_word_vector"],x["desc_word_vector"]),\
                                             axis=1)
prod_desc['cos_attr_word'] = prod_desc.apply(lambda x: spatial.distance.cosine(x["search_word_vector"],x["attr_word_vector"]),\
                                             axis=1)



In [ ]:
for i in range(0,len(prod_desc['search_term'])):
    if np.isnan(prod_desc["tfidf_search_word_vector"][i][1]):
        prod_desc.drop(i,inplace=True)
prod_desc = prod_desc.reset_index(drop=True)


for i in range(0,len(prod_desc['search_term'])):
    if np.isnan(prod_desc["tfidf_attr_word_vector"][i][1]):
        prod_desc.drop(i,inplace=True)
prod_desc = prod_desc.reset_index(drop=True)

In [ ]:
prod_desc['tfidf_title_word_vector'] = prod_desc['product_title'].apply\
(lambda x: tfidfFeatureVec(x,model,100,tfidf_dict,"word2vec"))
prod_desc['tfidf_desc_word_vector'] = prod_desc['product_description'].apply\
(lambda x: tfidfFeatureVec(x,model,100,tfidf_dict,"word2vec"))
prod_desc['tfidf_search_word_vector'] = prod_desc['search_term'].apply\
(lambda x: tfidfFeatureVec(x,model,100,tfidf_dict,"word2vec"))
prod_desc['tfidf_attr_word_vector'] = prod_desc['product_attribute_concat'].apply\
(lambda x: tfidfFeatureVec(x,model,100,tfidf_dict,"word2vec"))


prod_desc['tfidf_cos_title_word'] = prod_desc.apply(lambda x: spatial.distance.cosine(x["tfidf_search_word_vector"],\
                                                                        x["tfidf_title_word_vector"]),axis=1)
prod_desc['tfidf_cos_desc_word'] = prod_desc.apply(lambda x: spatial.distance.cosine(x["tfidf_search_word_vector"],\
                                                                           x["tfidf_desc_word_vector"]),axis=1)
prod_desc['tfidf_cos_attr_word'] = prod_desc.apply(lambda x: spatial.distance.cosine(x["tfidf_search_word_vector"],\
                                                                           x["tfidf_attr_word_vector"]),axis=1)

#lsa vectors
prod_desc['title_lsa_vector'] = prod_desc['product_title'].apply(lambda x: makeFeatureVec(x,vectorizer,100,"lsa"))
prod_desc['desc_lsa_vector'] = prod_desc['product_description'].apply(lambda x: makeFeatureVec(x,vectorizer,100,"lsa"))
prod_desc['search_lsa_vector'] = prod_desc['search_term'].apply(lambda x: makeFeatureVec(x,vectorizer,100,"lsa"))
prod_desc['attr_lsa_vector'] = prod_desc['product_attribute_concat'].apply(lambda x: makeFeatureVec(x,vectorizer,100,"lsa"))



In [ ]:
prod_desc['cos_title_lsa'] = prod_desc.apply(lambda x: spatial.distance.cosine(x["search_lsa_vector"],x["title_lsa_vector"]),\
                                             axis=1)
prod_desc['cos_desc_lsa'] = prod_desc.apply(lambda x: spatial.distance.cosine(x["search_lsa_vector"],x["desc_lsa_vector"]),\
                                            axis=1)
prod_desc['cos_attr_lsa'] = prod_desc.apply(lambda x: spatial.distance.cosine(x["search_lsa_vector"],x["attr_lsa_vector"]),\
                                            axis=1)

prod_desc['tfidf_title_lsa_vector'] = prod_desc['product_title'].apply\
(lambda x: tfidfFeatureVec(x,vectorizer,100,tfidf_dict,"lsa"))
prod_desc['tfidf_desc_lsa_vector'] = prod_desc['product_description'].apply\
(lambda x: tfidfFeatureVec(x,vectorizer,100,tfidf_dict,"lsa"))
prod_desc['tfidf_search_lsa_vector'] = prod_desc['search_term'].apply\
(lambda x: tfidfFeatureVec(x,vectorizer,100,tfidf_dict,"lsa"))
prod_desc['tfidf_attr_lsa_vector'] = prod_desc['product_attribute_concat'].apply\
(lambda x: tfidfFeatureVec(x,vectorizer,100,tfidf_dict,"lsa"))

prod_desc['tfidf_cos_title_lsa'] = prod_desc.apply(lambda x: spatial.distance.cosine(x["tfidf_search_lsa_vector"],\
                                                                        x["tfidf_title_lsa_vector"]),axis=1)
prod_desc['tfidf_cos_desc_lsa'] = prod_desc.apply(lambda x: spatial.distance.cosine(x["tfidf_search_lsa_vector"],\
                                                                           x["desc_lsa_vector"]),axis=1)
prod_desc['tfidf_cos_attr_lsa'] = prod_desc.apply(lambda x: spatial.distance.cosine(x["tfidf_search_lsa_vector"],\
                                                                           x["tfidf_attr_lsa_vector"]),axis=1)

#n-grams
prod_desc["bigram_title"]=prod_desc.apply(lambda x: ngram_count(x["search_term"],x["product_title"],2),axis=1)
prod_desc["bigram_desc"]=prod_desc.apply(lambda x: ngram_count(x["search_term"],x["product_description"],2),axis=1)
prod_desc["bigram_attr"]=prod_desc.apply(lambda x: ngram_count(x["search_term"],x["product_attribute_concat"],2),axis=1)

prod_desc["trigram_title"]=prod_desc.apply(lambda x: ngram_count(x["search_term"],x["product_title"],3),axis=1)
prod_desc["trigram_desc"]=prod_desc.apply(lambda x: ngram_count(x["search_term"],x["product_description"],3),axis=1)
prod_desc["trigram_attr"]=prod_desc.apply(lambda x: ngram_count(x["search_term"],x["product_attribute_concat"],3),axis=1)



In [ ]:
prod_desc['char_count'] = prod_desc['search_term'].apply(lambda x: sum(char.isalpha() for char in str(x)))
prod_desc['digits_perc'] = (prod_desc['search_term'].apply(lambda x: sum(char.isdigit() for char in str(x)))*100)/ \
                        prod_desc['search_term'].apply(lambda x: len(str(x).replace(" ","")))
prod_desc['color_match'] = prod_desc.apply(lambda x: match_query(x['product_color'],x['search_term']),axis=1)
prod_desc['brand_match'] = prod_desc.apply(lambda x: match_query(x['product_brand'],x['search_term'],),axis=1)
prod_desc['products_per_query'] = prod_desc.groupby("search_term")['product_uid'].count().reset_index(drop=True)
prod_desc['query_per_product'] = prod_desc.groupby("product_uid")['search_term'].count().reset_index(drop=True)




In [ ]:
prod_desc.to_csv('features.csv')